In [1]:
!pip install transformers gradio pandas openpyxl underthesea


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.4/978.4 kB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.6 MB/s eta 0:00:00


In [ ]:
import torch
import pandas as pd
import gradio as gr
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from google.colab import drive
from underthesea import word_tokenize # Cần tách từ cho PhoBERT

# 1. Kết nối Google Drive
drive.mount('/content/drive')

# ======================================================
# 2. CẤU HÌNH & LOAD MODEL
# ======================================================
MODEL_PATH = "/content/drive/MyDrive/Data - NLP/Final/phobert_model_download"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"⏳ Đang load model từ: {MODEL_PATH} ...")
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)
    model.to(device)
    print(f"✅ Load model thành công trên {device}!")
except Exception as e:
    print(f"❌ Lỗi load model: {e}")
    print("👉 Hãy kiểm tra kỹ đường dẫn MODEL_PATH.")

# Map nhãn
label_map = {0: 'Tiêu cực', 1: 'Bình thường', 2: 'Tích cực'}

# ======================================================
# 3. HÀM XỬ LÝ DỰ ĐOÁN
# ======================================================

def predict_text(text):
    """Hàm dự đoán cho 1 câu văn bản"""
    if not text or str(text).strip() == "":
        return "Không rõ", 0.0

    # 1. Tách từ (QUAN TRỌNG VỚI PHOBERT)
    # text: "Học sinh đi học" -> "Học_sinh đi học"
    segmented_text = word_tokenize(str(text), format="text")

    # 2. Tokenize & Đẩy vào GPU
    inputs = tokenizer(segmented_text, return_tensors="pt", truncation=True, padding=True, max_length=128).to(device)

    # 3. Dự đoán
    with torch.no_grad():
        outputs = model(**inputs)
        probs = outputs.logits.softmax(dim=1)

    pred_idx = torch.argmax(probs).item()
    confidence = probs.max().item()

    return label_map[pred_idx], confidence

# ======================================================
# 4. HÀM XỬ LÝ FILE EXCEL
# ======================================================
def analyze_excel(file_obj):
    # Đọc file Excel
    try:
        df = pd.read_excel(file_obj.name)
    except Exception as e:
        return None, f"Lỗi đọc file: {str(e)}"

    # Tìm cột chứa văn bản (Ưu tiên cột tên 'text', 'content', 'comment', hoặc lấy cột đầu tiên)
    target_col = None
    possible_names = ['text', 'content', 'comment', 'review', 'ý kiến', 'nội dung']

    # Check xem có cột nào trùng tên gợi ý không
    for col in df.columns:
        if col.lower() in possible_names:
            target_col = col
            break

    # Nếu không tìm thấy, lấy cột đầu tiên
    if target_col is None:
        target_col = df.columns[0]

    print(f"Đang xử lý cột: {target_col}")

    # Tạo list để chứa kết quả
    predictions = []
    confidences = []

    # Duyệt qua từng dòng để dự đoán (Có thể tối ưu bằng batch nhưng loop cho dễ hiểu)
    for text in df[target_col]:
        label, conf = predict_text(text)
        predictions.append(label)
        confidences.append(f"{conf:.2%}")

    # Ghi kết quả vào DataFrame
    df['Sentiment_Label'] = predictions
    df['Confidence'] = confidences

    # Lưu file kết quả ra bộ nhớ tạm
    output_path = "result_sentiment.xlsx"
    df.to_excel(output_path, index=False)

    return output_path, df.head(10) # Trả về đường dẫn file và bảng xem trước

# ======================================================
# 5. HÀM CHATBOT (Cập nhật dùng hàm predict_text)
# ======================================================
def chat_response(message, history):
    label, conf = predict_text(message)
    conf_str = f"{conf:.0%}"

    if "Tiêu cực" in label:
        return f"Dự báo: {label} ({conf_str})\nBot: Có vẻ tiêu cực nhỉ, bạn nên cẩn thận."
    elif "Tích cực" in label:
        return f"Dự báo: {label} ({conf_str})\nBot: Tín hiệu tốt đấy!"
    else:
        return f"Dự báo: {label} ({conf_str})\nBot: Mọi thứ đang ở mức trung bình."

# ======================================================
# 6. GIAO DIỆN GRADIO (TAB)
# ======================================================
with gr.Blocks(theme="soft", title="PhoBERT Sentiment App") as demo:
    gr.Markdown("# 🤖 Ứng dụng Phân tích cảm xúc với PhoBERT")

    with gr.Tabs():
        # TAB 1: Chatbot
        with gr.TabItem("💬 Chat Trực tiếp"):
            gr.ChatInterface(fn=chat_response, examples=["Cổ phiếu này sập sàn rồi", "Hôm nay trời đẹp quá"])

        # TAB 2: Excel Upload
        with gr.TabItem("📂 Phân tích file Excel"):
            gr.Markdown("Tải lên file Excel (.xlsx). Code sẽ tự động lấy cột dữ liệu đầu tiên hoặc cột tên 'text/content' để phân tích.")

            with gr.Row():
                inp_file = gr.File(label="Tải file Excel lên", file_types=[".xlsx"])
                btn_submit = gr.Button("🚀 Phân tích ngay", variant="primary")

            with gr.Row():
                out_file = gr.File(label="Tải xuống kết quả")
                out_dataframe = gr.Dataframe(label="Xem trước 10 dòng đầu", interactive=False)

            # Sự kiện bấm nút
            btn_submit.click(fn=analyze_excel, inputs=inp_file, outputs=[out_file, out_dataframe])

# Chạy App
demo.launch(share=True, debug=True)

Mounted at /content/drive
⏳ Đang load model từ: /content/drive/MyDrive/Data - NLP/Final/phobert_model_download ...
✅ Load model thành công trên cuda!


/tmp/ipython-input-3729031170.py:119: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme="soft", title="PhoBERT Sentiment App") as demo:
/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://3cc0ec15f6a826c2d5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
